## **Text classification (Sentiment analysis)**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) positive 0.8466
2) neutral 0.1458
3) negative 0.0076


In [8]:
texts = ["Good night 😊", "I love you", "fuck"]

outputs = []
preds = []

for text in texts:
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  l = labels[ranking[0]]
  print(f"label: {l}")
  preds.append(l)
  outputs.append(output)

label: positive
label: positive
label: negative


In [9]:
preds

['positive', 'positive', 'negative']

## **Text generation (GPT2)**

In [10]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, but what I'm really doing is making a human-readable document. There are other languages, but those are"},
 {'generated_text': "Hello, I'm a language model, not a syntax model. That's why I like it. I've done a lot of programming projects.\n"},
 {'generated_text': "Hello, I'm a language model, and I'll do it in no time!\n\nOne of the things we learned from talking to my friend"},
 {'generated_text': "Hello, I'm a language model, not a command line tool.\n\nIf my code is simple enough:\n\nif (use (string"},
 {'generated_text': "Hello, I'm a language model, I've been using Language in all my work. Just a small example, let's see a simplified example."}]

In [11]:
templates_positive = ["I'm so happy, look: ", "A sunny day "]
templates_negative = ["So sad that", "It is raining "]

In [15]:
n_positive = 10
n_per_start = 5 #len(templates_positive) // n_positive

for start in templates_positive:
  res = list(generator(start, max_length=30, num_return_sequences=n_per_start))
  print(res)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I'm so happy, look: \ue60c \ue637\n\nHe has won more than 100 caps for England and he has become"}, {'generated_text': "I'm so happy, look: !!\n\n\nI'm happy because I didn't have a good night's sleep because my daughter is crazy about"}, {'generated_text': "I'm so happy, look: \xa0I got that shot because I'm still looking forward to it, not because I'm looking forward to the"}, {'generated_text': 'I\'m so happy, look: \xa0I\'m doing fine, good."\nSomewhere along the way I remembered the time I sat in'}, {'generated_text': "I'm so happy, look:  idle and depressed, and so happy that I can finally write a book!\nWe all know how bad things"}]
[{'generated_text': 'A sunny day !!\nMight use new one soon...\nLove the look!!'}, {'generated_text': 'A sunny day __________________ Last edited by BofG; 05-05-2016 at 08:30 AM.'}, {'generated_text': 'A sunny day ~~ ~~\n\nBeneath the bright morning of October 21, 1945, my wife and I made a small trip home'}, {'generated_text': '